# Pulling Stock Data
Python notebook to pull stock data for D3 visualization: https://bl.ocks.org/mbostock/1256572

## Sourcing Data
Let's start by using [Quandl](https://www.quandl.com) for the stock data:

In [ ]:
import quandl
quandl.ApiConfig.api_key = "Byjzu4U8rmR1iEhZnp7V"

Let's pull a single stock to explore the data format:

In [ ]:
data = quandl.get("WIKI/AAPL", rows=5)

What format is the data?

In [ ]:
print type(data)

Let's peek at the structure of this dataframe

In [ ]:
data

We are interested in monthly values of the `Close` price which is the 4th column, so let's see if we can extract just that using `.4` from their [API documentation](https://docs.quandl.com/docs/time-series-2):

In [ ]:
data = quandl.get("WIKI/AAPL.4", rows=5, collapse='monthly')
data

Great but we want to show the last 10 years, so let's work out the start date based on today:

In [ ]:
import datetime
start_date = (datetime.datetime.now() - datetime.timedelta(days=10*365)).strftime('%Y-%m-01')
start_date

In [ ]:
data = quandl.get("WIKI/AAPL.4", rows=120, collapse='monthly', start_date=start_date)
data.head(5)

Let's do a quick visualization to check the data looks right:

In [ ]:
%matplotlib inline
ax = data.plot()

Uh-oh! Looks like we have a glitch. Why is there a big drop in AAPL stock in 2014? A quick Google shows they applied a [Stock Split](https://www.washingtonpost.com/news/the-switch/wp/2014/06/09/apples-stock-price-just-dropped-more-than-500-a-share-but-dont-panic/). 

We'll need to use the `Adjusted Close` price (column 11) from Quandl to account for that:

In [ ]:
data = quandl.get("WIKI/AAPL.11", rows=120, collapse='monthly', start_date=start_date)
ax = data.plot()

Great! Let's get it ready for D3.js

## Scrubbing Data

The data format we need for the D3.js showcase is:

```
symbol,date,price
MSFT,Jan 2000,39.81
MSFT,Feb 2000,36.35
MSFT,Mar 2000,43.22
MSFT,Apr 2000,28.37
MSFT,May 2000,25.45```

Let's choose a few symbols and pull the data we need:

In [ ]:
symbols = ['WIKI/MSFT.11', 'WIKI/AAPL.11', 'WIKI/IBM.11', 'WIKI/GOOGL.11']
data = quandl.get(symbols, rows=120, collapse='monthly', start_date=start_date)
data.head(5)

Let's rename the columns to just their symbol name by removing `WIKI/` and ` - Close` from each column name. We can iterate over the columns using the `for col in data.columns` syntax:

In [ ]:
data.columns = [col.replace(' - Adj. Close','').replace('WIKI/','') for col in data.columns]
data.head(5)

Let's have another quick look:

In [ ]:
ax = data.plot()

In [ ]:
datalist=data.unstack()
datalist.head(10)

Almost there! We just need the CSV format now:

In [ ]:
csv = datalist.to_csv(header=True, index_label=['symbol','date','price'], date_format='%b %Y', index=True)
csv = csv.replace("price,0","price") # remove addition of ',0' on first line
print csv

Ok let's make a downloadable file version:

In [ ]:
import base64
from IPython.display import HTML

b64 = base64.b64encode(csv.encode())
payload = b64.decode()
html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{filename}</a>'
html = html.format(payload=payload,title="stocks.csv",filename="stocks.csv")
HTML(html)
